In [1]:
import requests
import json 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import functools

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests

#### Autentication

In [2]:
credentials = {"username":"","password":""}
username = credentials["username"]
URL = f'https://api.brandwatch.com/oauth/token?username={username}&grant_type=api-password&client_id=brandwatch-api-client'
data_ = {"password":credentials["password"]}

response_access_token = requests.get(URL, params = data_).json()

In [3]:
URL = f'https://api.brandwatch.com/user'
data_ = {"access_token":response_access_token["access_token"]}

response = requests.get(URL, params = data_).json()

In [4]:
project_id = "1998324599"
URL = f'https://api.brandwatch.com/projects/{project_id}/queries/summary'
data_ = {"access_token":response_access_token["access_token"]}

response = requests.get(URL, params = data_).json()

#### Creation

Specifying Languages

In [5]:
URL = 'https://api.brandwatch.com/languages'
response = requests.get(URL, params = data_).json()
indicator= False
language_list=[]
while indicator == False:
    x=input("Enter name of a language with the first letter as capital or STOP to stop ")
    if x!= "STOP":
        language_list.append(x)
    else:
        indicator= True
        
codes=[]
for i in response["results"]:
    if i["name"] in language_list:
        codes.append(i["code"])

Enter name of a language with the first letter as capital or STOP to stop Russian
Enter name of a language with the first letter as capital or STOP to stop Ukrainian
Enter name of a language with the first letter as capital or STOP to stop STOP


Specifying Countries

In [7]:
indicator= False
countries=[]
while indicator == False:
    x=input("Enter name of a country or STOP to stop ")
    if x!= "STOP":
        countries.append(x)
    else:
        indicator= True
        
country_codes=[]
for i in countries :
    url = f'https://api.brandwatch.com/locations?type=country&prefix={i}'
    response = requests.get(url, params = data_).json()
    country_codes.append(response["results"][0]["id"])

Enter name of a country or STOP to stop Russia
Enter name of a country or STOP to stop Ukraine
Enter name of a country or STOP to stop STOP


In [8]:
# Validation
URL=  'https://api.brandwatch.com/query-validation'
data_ = {"access_token":response_access_token["access_token"]}

param_data={
    "booleanQuery": "Conflict",
    "languages" : codes,
    "imageFilter": {
        "imageAnalysisType": "images-or-keywords",
        "logoIds": [
            1234
        ]
    },
    "locationFilter": {
        "includedLocations": country_codes
    }
}

response = requests.post(URL,params = data_,json=param_data ).json()
response

{'booleanQuery': 'Conflict',
 'languages': ['ru', 'uk'],
 'locationFilter': {'includedLocations': ['RUS', 'UKR'],
  'excludedLocations': []},
 'imageFilter': {'logoIds': [1234],
  'objectIds': [],
  'imageAnalysisType': 'images-or-keywords'},
 'audienceLists': None,
 'contentSources': ['blog',
  'facebook',
  'forum',
  'instagram',
  'instagram_public',
  'linkedin',
  'news',
  'qq',
  'reddit',
  'review',
  'tiktok',
  'tumblr',
  'twitter',
  'youtube'],
 'errors': [],
 'issues': [],
 'samplePercentage': None,
 'queryLimitUsage': 1,
 'exclusionSnippets': None}

In [9]:
# Creation
URL='https://api.brandwatch.com/projects/1998159537/queries/'
data_ = {"access_token":response_access_token["access_token"]}
param_data={
  "languages": codes,
  "booleanQuery": "Conflict",
  "name": "Conflict",
  "startDate": "2021-02-28",
  "contentSources": [
    "blog",
    "twitter",
    "forum"
  ],
  "lockedQuery": "true",
  "audienceLists": [],
  "imageFilter": {
      
    "logoIds": [
      1234
    ],
    "objectIds": [
      1234
    ]
  },
  "locationFilter": {
    "includedLocations": country_codes
  }
}
response = requests.post(URL,params = data_,json=param_data ).json()

In [10]:
response

{'errors': [{'code': 300,
   'message': 'You do not have write permissions to the project'}]}

#### Retrieval 

In [51]:
URL = f'https://api.brandwatch.com/projects/{project_id}/queries/summary'
data_ = {"access_token":response_access_token["access_token"]}

response = requests.get(URL, params = data_).json()

In [52]:
for result in response["results"]:
    query_id = result["id"]
    name = result["name"]
    print(name, "| Query ID:", query_id)

Australian Dollar | Query ID: 2000889167
Brazilian Real | Query ID: 2000853631
Hong Kong Dollar | Query ID: 2000886195
Mexican Peso | Query ID: 2000853632
South African Rand | Query ID: 2000853633
Turkish Lira | Query ID: 2000853629


In [53]:
name = "Australian Dollar"
query_id = "2000889167"
response

{'resultsTotal': 6,
 'resultsPage': -1,
 'resultsPageSize': -1,
 'results': [{'id': 2000889167,
   'name': 'Australian Dollar',
   'displayName': None,
   'type': 'monitor',
   'creationDate': '2022-02-04T18:17:34.189+0000',
   'lastModificationDate': '2022-02-04T18:17:34.209+0000',
   'lastModifiedUsername': 'cp126@nyu.edu',
   'lockedQuery': False,
   'lockedByUsername': None,
   'createdByWizard': True,
   'languages': ['en', 'mi'],
   'contentSources': ['news',
    'forum',
    'youtube',
    'qq',
    'twitter',
    'review',
    'facebook',
    'reddit',
    'tumblr',
    'instagram_public',
    'instagram',
    'blog'],
   'imageFilter': None,
   'state': 'active',
   'audienceLists': [],
   'sentimentClassifierVersion': 3,
   'languageAgnostic': False},
  {'id': 2000853631,
   'name': 'Brazilian Real',
   'displayName': None,
   'type': 'monitor',
   'creationDate': '2022-01-08T01:38:25.564+0000',
   'lastModificationDate': '2022-02-01T09:00:14.260+0000',
   'lastModifiedUserna

In [54]:
start = "2021-02-28"
end = "2022-02-28"
URL = f'https://api.brandwatch.com/projects/{project_id}/data/volume/sentiment/days?queryId={query_id}&startDate={start}&endDate={end}'

data_ = {"access_token":response_access_token["access_token"]}

response = requests.get(URL, params = data_).json()

In [55]:
output = []
for result in response["results"]:
    value = result["name"]
    df = pd.DataFrame(result["values"])
    df = df[["name", "value"]].copy()
    df.rename(columns = {"name":"timestamp", "value": f"{value} volume"}, inplace = True)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace = True)
    output.append(df)

In [56]:
df = functools.reduce(lambda x,y: pd.merge(x,y,left_index = True, right_index = True), output)
df["media volume"] = df.sum(axis = 1)
df.index.name = name

In [57]:
df.reset_index(inplace=True)
df.rename(columns = {"Australian Dollar":"date"},inplace=True)
df

,date,positive volume,neutral volume,negative volume,media volume
0,2021-02-28,458,1482,577,2517
1,2021-03-01,723,3513,1065,5301
2,2021-03-02,777,3643,1046,5466
3,2021-03-03,777,3241,925,4943
4,2021-03-04,898,3286,852,5036
...,...,...,...,...,...
360,2022-02-23,228,5296,525,6049
361,2022-02-24,166,4587,571,5324
362,2022-02-25,171,4048,438,4657
363,2022-02-26,169,2006,270,2445


#### FX Data

In [47]:
BASE_URL = "https://www.alphavantage.co/query"
from_currency="EUR"
to_currency="USD"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-04,1.09312
1,2022-03-03,1.10659
2,2022-03-02,1.11186
3,2022-03-01,1.11239
4,2022-02-28,1.12175
...,...,...
95,2021-10-22,1.16470
96,2021-10-21,1.16250
97,2021-10-20,1.16480
98,2021-10-19,1.16286


As we did not have write permissions I downloaded the data for conflict

In [14]:
conflict=pd.read_csv("conflict.csv")
conflict.date=conflict.date.astype("datetime64")
conflict

,date,volume
0,2022-01-29,14325
1,2022-01-30,80902
2,2022-01-31,85266
3,2022-02-01,85196
4,2022-02-02,92291
5,2022-02-03,100882
6,2022-02-04,92285
7,2022-02-05,75837
8,2022-02-06,66046
9,2022-02-07,81750


#### Analysis

#### EUR-USD

In [49]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])
correl_test.corr()

,volume,price
volume,1.000000,-0.566529
price,-0.566529,1.000000


In [50]:
correl_test

,date,volume,price
0,2022-01-31,85266,1.12334
1,2022-02-01,85196,1.12704
2,2022-02-02,92291,1.13022
3,2022-02-03,100882,1.14400
4,2022-02-04,92285,1.14480
5,2022-02-07,81750,1.14412
6,2022-02-08,85987,1.14153
7,2022-02-09,91615,1.14230
8,2022-02-10,103735,1.14270
9,2022-02-11,102925,1.13457


Correlation with Lead and Lag

Offset by 7 days

In [51]:
offset = 7
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      


In [52]:
df_offset

,date,volume,price
0,2022-01-31,85266,1.14230
1,2022-02-01,85196,1.14270
2,2022-02-02,92291,1.13457
3,2022-02-03,100882,1.13052
4,2022-02-04,92285,1.13580
5,2022-02-07,81750,1.13721
6,2022-02-08,85987,1.13592
7,2022-02-09,91615,1.13203
8,2022-02-10,103735,1.13102
9,2022-02-11,102925,1.13230


In [53]:
df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,-0.863736
price,-0.863736,1.000000


offset by 1 day

In [54]:
offset = 1
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      
df_offset

,date,volume,price
0,2022-01-31,85266,1.12704
1,2022-02-01,85196,1.13022
2,2022-02-02,92291,1.14400
3,2022-02-03,100882,1.14480
4,2022-02-04,92285,1.14412
5,2022-02-07,81750,1.14153
6,2022-02-08,85987,1.14230
7,2022-02-09,91615,1.14270
8,2022-02-10,103735,1.13457
9,2022-02-11,102925,1.13052


In [55]:
df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,-0.541353
price,-0.541353,1.000000


In [56]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: 1.303647
p-value: 0.996629
Critical values:
	1%: -4.069
	5%: -3.127
	10%: -2.702


In [57]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -2.441374
p-value: 0.130398
Critical values:
	1%: -4.223
	5%: -3.189
	10%: -2.730


In [58]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.114286
volume,-0.114286,1.000000


In [59]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [60]:
grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])

,volume,price
volume,1.0000,0.34
price,0.0844,1.00


#### GBP-CHF

In [15]:
BASE_URL = "https://www.alphavantage.co/query"
from_currency="GBP"
to_currency="CHF"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-09,1.22060
1,2022-03-08,1.21672
2,2022-03-07,1.21274
3,2022-03-04,1.21391
4,2022-03-03,1.22445
...,...,...
95,2021-10-27,1.26130
96,2021-10-26,1.26610
97,2021-10-25,1.26530
98,2021-10-22,1.25959


In [16]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])
correl_test.corr()

,volume,price
volume,1.00000,-0.56483
price,-0.56483,1.00000


#### offset by 1 day

In [17]:

offset = 1
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      

df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,-0.396992
price,-0.396992,1.000000


#### Offset by 7 days

In [18]:
offset = 7
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      
df_offset

df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,-0.727473
price,-0.727473,1.000000


In [19]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: 0.954689
p-value: 0.993764
Critical values:
	1%: -3.859
	5%: -3.042
	10%: -2.661


In [23]:
result = adfuller(correl_test["price"].pct_change(periods=5).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: 2.422344
p-value: 0.999020
Critical values:
	1%: -4.473
	5%: -3.290
	10%: -2.772


In [22]:
df_diff = correl_test[["price","volume"]].pct_change(periods=5).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.785294
volume,-0.785294,1.000000


In [68]:
grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])

,volume,price
volume,1.0000,0.1862
price,0.3618,1.0000


#### Deletion 

In [ ]:
URL ="https://api.brandwatch.com/projects/398748937/queries/11554" # this is just a sample
response= requests.delete(URL,params = data_)
